## Azure Pipeline Deployment

In [ ]:
!pip install kfp

In [ ]:
import kfp
import os
import calendar
import time
from kubernetes import client

In [ ]:
component_root_train = 'components/train_upload_model/'
component_root_register = 'components/register_model/'
component_root_deploy = 'components/deploy_model/'

In [ ]:
azure_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
azure_register_op = kfp.components.load_component_from_file(os.path.join(component_root_register, 'component.yaml'))
azure_deploy_op = kfp.components.load_component_from_file(os.path.join(component_root_deploy, 'component.yaml'))

In [ ]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

In [ ]:
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)

In [ ]:
def azure_pipeline():
    
    #Define task for training BLERSSI data
    azure_train_task = azure_train_op(azure_storage_account_name='',
                                      azure_storage_account_key='')
    azure_train_task.add_volume(nfs_volume)
    azure_train_task.add_volume_mount(nfs_volume_mount)
    
    #Define task for registering BLERSSI model on Azure ML Studio
    azure_register_task = azure_register_op(workspace_name='',
                                           subscription_id='',
                                           resource_group='',
                                           model_name='model2',
                                           tenant_id='',
                                           service_principal_id='',
                                           service_principal_password='')
    
    azure_register_task.add_volume(nfs_volume)
    azure_register_task.add_volume_mount(nfs_volume_mount)
    azure_register_task.after(azure_train_task)
    
    #Define Task for deploying BLERSSI model on Azure ML Studio
    azure_deploy_task = azure_deploy_op(workspace_name='',
                                       subscription_id='',
                                       resource_group='',
                                       model_name='model2',
                                       service_name='service2',
                                       tenant_id='',
                                       service_principal_id='',
                                       service_principal_password='')

    azure_deploy_task.after(azure_register_task)

In [ ]:
#Create a pipeline run
kfp.Client().create_run_from_pipeline_func(azure_pipeline, arguments={})

In [ ]:
import os
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace
svc_pr_password = os.environ.get("AZUREML_PASSWORD")

svc_pr = ServicePrincipalAuthentication(
tenant_id="",
service_principal_id="",
service_principal_password="")

ws = Workspace(
subscription_id="",
resource_group="",
workspace_name="",
auth=svc_pr
)

In [ ]:
from azureml.core.webservice import Webservice
service = Webservice(workspace=ws, name='service2')
print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
service.get_logs()

In [ ]:
X=[[1,2,3,4,5,6,7,8,9,3,4,3,6]]
#print(type(X))
import json
import numpy as np
import base64

#data={"data": X}
test_samples1 = json.dumps({"data": X})

test_samples1
print(type(test_samples1))
test_samples1 = bytes(test_samples1, encoding='utf8')

print(test_samples1)
print(type(test_samples1))
service.run(input_data=test_samples1)